<a href="https://colab.research.google.com/github/theantigone/Prompt-Engineering-for-In-Context-Learning/blob/master/notebooks/1.0-qh-prompt-engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
  # pip install openai
!pip3 install openai
!pip3 install sacrebleu               # text BLEU (reference-free variant)
!git clone -q https://github.com/microsoft/CodeXGLUE.git

In [ ]:
"""Run this model in Python

> pip install openai
"""
import os
from openai import OpenAI
import pandas as pd

# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
with open("token.txt", "r") as f:
    GITHUB_TOKEN = f.read().strip()

# Set it as an environment variable
# os.environ["GITHUB_TOKEN"] = "X"


# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    # api_key=os.environ["GITHUB_TOKEN"],
    api_key=GITHUB_TOKEN,
)

# List of available models
model_choices = ["gpt-4.1" , "Codestral-2501", "gpt-4.1-mini", "gpt-4.1-nano", "gpt-4o-mini", "mistral-small-2503", "Mistral-small", "mistral-nemo", "ministral-3b"]
#DeepSeek is not allowed on gov owened machines

In [ ]:
# Zero-shot Prompt
zero_shot_tasks = [
    {
        "content": "Summarize the functionality of the following method:",
        "code": '''
public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}
'''
    },
    {
        "content": "Identify and fix the off-by-one error in this function:",
        "code": '''
def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
'''
    },
    {
        "content": "Classify the type of bug in the following C++ function:",
        "code": '''
int* getArray(int size) {
    int arr[size]; // Warning: local array
    return arr; // Bug: returning pointer to local variable
}
'''
    },
    {
        "content": "Complete the function using regex to validate basic email addresses:",
        "code": '''
def is_valid_email(email):
    # TODO: Complete using regex
    pass
'''
    },
    {
        "content": "Create a ‘/greet/<username>’ endpoint that returns a JSON greeting:",
        "code": '''
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    # TODO: Return a JSON greeting
    pass
'''
    },
    {
        "content": "Write the schema for a review app with users, books, and reviews:",
        "code": '''
-- TODO: Design schema with appropriate keys and constraints
-- Tables: users(id, name), books(id, title), reviews(id, user_id, book_id, rating)
'''
    },
    {
        "content": "Identify any null dereference risk:",
        "code": '''
public int getLength(String s) {
    return s.length(); // What if s is null?
}
'''
    },
    {
        "content": "Improve the parser to support quoted fields:",
        "code": '''
def parse_csv_line(line):
    return line.split(',')  # Incomplete: doesn’t handle quoted fields
'''
    },
    {
        "content": "Convert the data class to a REST API using Ktor:",
        "code": '''
data class Product(val id: Int, val name: String, val price: Double)

# TODO: Create GET and POST endpoints using Ktor
'''
    },
    {
        "content": "Write a brief summary of the function:",
        "code": '''
def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
'''
    },
    {
        "content": "Write a prompt that could generate the code:",
        "code": '''
# This function checks if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) +1):
        if n % i == 0:
            return False
    return True
'''
    },
    {
        "content": "Fix the bug when input is 0:",
        "code": '''
def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result
'''
    },
    {
        "content": "Implement node deletion by value:",
        "code": '''
struct Node {
    int data;
    struct Node* next;
};

void deleteNode(struct Node** head, int key) {
    // TODO: Implement node deletion
}
'''
    },
    {
        "content": "Complete the recursive function for Fibonacci:",
        "code": '''
def fibonacci(n):
    # TODO: Base cases and recursive call
    pass
'''
    },
    {
        "content": "Complete the class constructor:",
        "code": '''
class Person:
    def __init__(self):
        # TODO: Add name, age, and optional email
        pass
'''
    },
    {
        "content": "Complete the binary search implementation:",
        "code": '''
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left <= right) {
        int mid = (left + right) / 2;
        // TODO: Compare and adjust bounds
    }
    return -1;
}
'''
    },
    {
        "content": "Resolve inconsistency between function name and logic:",
        "code": '''
// Supposed to return true if x is even
bool isOdd(int x) {
    return x % 2 == 0; // Logic contradicts function name
}
'''
    },
    {
        "content": "Identify and fix the bug:",
        "code": '''
function isEven(n) {
    return n % 2; // Returns 1 or 0, not true/false
}
'''
    },
    {
        "content": "Decompose the high-level comment/summary into logical steps:",
        "code": '''
// Function that validates an input, calculates square, and returns result
int process(int x) {
    if (x < 0) return -1;
    return x * x;
}
'''
    },
    {
        "content": "Complete the function based on intent:",
        "code": '''
def calculate_average(scores):
    total = 0
    # TODO: Complete to return average
    pass
'''
    },
    {
        "content": "Analyze the following utility script. Identify any potential logic issues or design flaws. Then, refactor it for better readability, correctness, and safety:",
        "code": '''
# utils.py - Script to parse and summarize numeric CSV files

import csv

def read_csv(filepath):
    with open(filepath, 'r') as f:
        return [row for row in csv.reader(f)]

def summarize_column(data, index):
    values = [float(row[index]) for row in data[1:]]  # skip header
    total = sum(values)
    avg = total / len(values)
    return total, avg

def main():
    filepath = 'data.csv'
    data = read_csv(filepath)
    total, avg = summarize_column(data, 1)
    print("Total:", total)
    print("Average:", avg)

if __name__ == '__main__':
    main()
'''
    },
    {
        "content": "Complete the following file-processing script. The goal is to clean each line, remove punctuation, and count word frequencies correctly:",
        "code": '''
# file_processor.py - Incomplete script for processing text files

import string

def load_file(filepath):
    with open(filepath, 'r') as f:
        return f.readlines()

def clean_line(line):
    # TODO: Remove punctuation and make lowercase
    pass

def count_words(lines):
    word_counts = {}
    for line in lines:
        clean = clean_line(line)
        for word in clean.split():
            word_counts[word] = word_counts.get(word, 0) + 1
    return word_counts

def main():
    filepath = 'input.txt'
    lines = load_file(filepath)
    counts = count_words(lines)
    for word, count in sorted(counts.items()):
        print(f"{word}: {count}")

if __name__ == '__main__':
    main()
'''
    }
]

chain_of_thought_tasks = [
    {
        "content": "Let's break down the method step-by-step. First, it initializes a frequency map. Then, it iterates through the list of words. For each word, it updates the frequency map by checking if the word is already present. After the loop, it returns the frequency map. Now, summarize the functionality of the following method in a concise manner.",
        "code": '''
public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}
'''
    },
    {
        "content": "Let's first analyze the loop. The range for i starts at start and ends at end - 1. This could cause an off-by-one error. Now, fix this so that the range includes the correct endpoints",
        "code": '''
def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
'''
    },
    {
        "content": "Let's walk through this function. The function declares a local array arr and returns a pointer to it. However, returning a pointer to a local variable is risky because the array's memory will be deallocated once the function exits. Identify the type of bug here.",
        "code": '''
int* getArray(int size) {
    int arr[size]; // Warning: local array
    return arr; // Bug: returning pointer to local variable
}
'''
    },
    {
        "content": "To validate an email address using regex, we need to check for a few key patterns: a non-empty username followed by the @ symbol, followed by a domain name and a valid top-level domain. Let's construct a regex pattern to match this.",
        "code": '''
def is_valid_email(email):
    # TODO: Complete using regex
    pass
'''
    },
    {
        "content": "First, think about the endpoint '/greet/<username>'. The username should be passed to the function greet(), which should return a greeting message in JSON format. Write the function that returns a JSON response with a greeting message.",
        "code": '''
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    # TODO: Return a JSON greeting
    pass
'''
    },
    {
        "content": "Let's start by defining the tables needed for this app. We need a table for users, with fields like id and name, and a table for books, with fields like id and title. Lastly, we need a table for reviews, which links users and books with a rating. Now, write the schema for this.",
        "code": '''
-- TODO: Design schema with appropriate keys and constraints
-- Tables: users(id, name), books(id, title), reviews(id, user_id, book_id, rating)
'''
    },
    {
        "content": "The function getLength() tries to access the length of a string. However, if the string is null, this will throw a NullPointerException. How can we handle this to avoid a null dereference?",
        "code": '''
public int getLength(String s) {
    return s.length(); // What if s is null?
}
'''
    },
    {
        "content": "Currently, the function splits each line by commas. However, if a field is quoted, it should be treated as a single value, even if it contains commas. Modify the function to handle quoted fields correctly.",
        "code": '''
def parse_csv_line(line):
    return line.split(',')  # Incomplete: doesn’t handle quoted fields
'''
    },
    {
        "content": "We need to expose a Product data class through a REST API using Ktor. Let's create a GET endpoint to retrieve product details and a POST endpoint to create new products.",
        "code": '''
data class Product(val id: Int, val name: String, val price: Double)

# TODO: Create GET and POST endpoints using Ktor
'''
    },
    {
        "content": "Let's break down this function. It splits the sentence into words and then reverses the order of the words. Finally, it joins the reversed words into a string and returns it. Now, summarize its purpose.",
        "code": '''
def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
'''
    },
    {
        "content": "Consider the comment that explains the function: 'This function checks if a number is prime.' The model needs to identify the steps to implement a prime-checking function. Let's walk through the logic and write the function.",
        "code": '''
# This function checks if a number is prime
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) +1):
        if n % i == 0:
            return False
    return True
'''
    },
    {
        "content": "Think about the issue when n = 0. The loop starts at 1, but it should handle the case where n = 0. Let's fix the loop to handle this case correctly.",
        "code": '''
def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result
'''
    },
    {
        "content": "To delete a node in a linked list, we need to find the node with the matching value and remove it. Consider the pointers to previous and next nodes in the list to ensure we correctly unlink the node. Now, implement the node deletion.",
        "code": '''
struct Node {
    int data;
    struct Node* next;
};

void deleteNode(struct Node** head, int key) {
    // TODO: Implement node deletion
}
'''
    },
    {
        "content": "Let's implement the base cases for the Fibonacci sequence. If n is 0 or 1, return n. Otherwise, recursively call the function for n-1 and n-2. Now, complete the function.",
        "code": '''
def fibonacci(n):
    # TODO: Base cases and recursive call
    pass
'''
    },
    {
        "content": "Think about the Person class. We need to initialize name, age, and an optional email field. Define the constructor to properly set these values.",
        "code": '''
class Person:
    def __init__(self):
        # TODO: Add name, age, and optional email
        pass
'''
    },
    {
        "content": "Let's write the logic for binary search. First, find the middle index. If the target is smaller than the middle element, adjust the right boundary. If it's larger, adjust the left boundary. Now, implement the binary search.",
        "code": '''
public int binarySearch(int[] arr, int target) {
    int left = 0, right = arr.length - 1;
    while (left <= right) {
        int mid = (left + right) / 2;
        // TODO: Compare and adjust bounds
    }
    return -1;
}
'''
    },
    {
        "content": "This function is supposed to check if a number is even, but it returns the result for odd numbers. Fix the logic to return true when the number is even.",
        "code": '''
// Supposed to return true if x is even
bool isOdd(int x) {
    return x % 2 == 0; // Logic contradicts function name
}
'''
    },
    {
        "content": "Let's first check the logic of the function. It returns 1 or 0 instead of true or false when checking if a number is even. Now, let's fix the return statement.",
        "code": '''
function isEven(n) {
    return n % 2; // Returns 1 or 0, not true/false
}
'''
    },
    {
        "content": "First, the function validates the input. Then, it calculates the square of the input if valid. Finally, it returns the result. Decompose the steps and describe the function's behavior.",
        "code": '''
// Function that validates an input, calculates square, and returns result
int process(int x) {
    if (x < 0) return -1;
    return x * x;
}
'''
    },
    {
        "content": "The function is likely intended to calculate an average of a list of scores. We need to sum the scores and then divide by the total number of scores. Complete the function to return the average.",
        "code": '''
def calculate_average(scores):
    total = 0
    # TODO: Complete to return average
    pass
'''
    },
    {
        "content": "Let’s analyze the script. The function summarize_column() assumes that the second column in the CSV contains numeric data. Check if the data is valid before performing calculations and refactor the code to handle edge cases.",
        "code": '''
# utils.py - Script to parse and summarize numeric CSV files

import csv

def read_csv(filepath):
    with open(filepath, 'r') as f:
        return [row for row in csv.reader(f)]

def summarize_column(data, index):
    values = [float(row[index]) for row in data[1:]]  # skip header
    total = sum(values)
    avg = total / len(values)
    return total, avg

def main():
    filepath = 'data.csv'
    data = read_csv(filepath)
    total, avg = summarize_column(data, 1)
    print("Total:", total)
    print("Average:", avg)

if __name__ == '__main__':
    main()
'''
    },
    {
        "content": "Let's clean up the lines by removing punctuation and converting to lowercase. Then, we'll split the lines into words and count their frequencies. Complete the script to handle file processing and word counting.",
        "code": '''
# file_processor.py - Incomplete script for processing text files

import string

def load_file(filepath):
    with open(filepath, 'r') as f:
        return f.readlines()

def clean_line(line):
    # TODO: Remove punctuation and make lowercase
    pass

def count_words(lines):
    word_counts = {}
    for line in lines:
        clean = clean_line(line)
        for word in clean.split():
            word_counts[word] = word_counts.get(word, 0) + 1
    return word_counts

def main():
    filepath = 'input.txt'
    lines = load_file(filepath)
    counts = count_words(lines)
    for word, count in sorted(counts.items()):
        print(f"{word}: {count}")

if __name__ == '__main__':
    main()
'''
    }
]


In [ ]:
tasks_few_shot = [
# 1 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example 1 (Java)
Code:
public int add(int a, int b){ return a + b; }
Summary:
Returns the sum of two integers.

Example 2 (Java)
Code:
public boolean isEmpty(List<?> list){ return list.size() == 0; }
Summary:
Checks if a list has no elements.

Now summarise, in one sentence, the functionality of the following method:""",
"code": '''
public Map<String,Integer> countWordFrequency(List<String> words){
    Map<String,Integer> freqMap = new HashMap<>();
    for(String word:words){
        freqMap.put(word,freqMap.getOrDefault(word,0)+1);
    }
    return freqMap;
}
'''
},
# 2 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Python)
Buggy code:
def inclusive_sum(a,b):
    total=0
    for i in range(a,b):
        total+=i
    return total
Fixed explanation:
Use range(a,b+1) or range(a,b) with b incremented.

Your task: identify and fix the off‑by‑one error in the next function, then show the corrected code:""",
"code": '''
def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
'''
},
# 3 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (C++)
Buggy function:
int* bad(){ int x=5; return &x; }
Bug classification:
Returning pointer to local (stack) variable  ➜  **dangling pointer / use‑after‑scope**.

Given the next C++ function, classify the main bug type in one phrase:""",
"code": '''
int* getArray(int size){
    int arr[size];
    return arr;
}
'''
},
# 4 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Python + regex)
Input regex: r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}$'
Explanation: username, '@', domain, '.', TLD 2+ letters.

Supply a regex‑based implementation that returns True for valid emails and False otherwise in the function below:""",
"code": '''
def is_valid_email(email):
    # TODO: Complete using regex
    pass
'''
},
# 5 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Flask)
@app.route('/hello/<name>')
def hello(name):
    return jsonify({'msg':f'Hello {name}!'})

Following the pattern, create a '/greet/<username>' endpoint that returns JSON {"greeting":"Hello, <username>!"}:""",
"code": '''
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    # TODO
    pass
'''
},
# 6 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (SQL schema snippet)
CREATE TABLE users(id SERIAL PRIMARY KEY, name TEXT);

Design tables users, books, reviews with proper PK‑FK relations and rating INT 1‑5:""",
"code": '''
-- your schema here
'''
},
# 7 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Java null‑check)
if(s == null) return 0; else return s.length();

Task: point out the null‑dereference risk in the following method and suggest a safe fix:""",
"code": '''
public int getLength(String s){
    return s.length();
}
'''
},
# 8 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Python CSV)
Use csv module or regex to keep quoted fields intact, e.g.
csv.reader([line])  ➜  list(row)

Rewrite the parser so it correctly handles lines with quotes:""",
"code": '''
def parse_csv_line(line):
    return line.split(',')  # broken
'''
},
# 9 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (Ktor)
get("/product/{id}"){ ... }
post("/product"){ ... }

Create equivalent GET and POST routes for the Product data class shown below:""",
"code": '''
data class Product(val id:Int, val name:String, val price:Double)
'''
},
# 10 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (summary)
`def double(x): return 2*x` ➜ "Returns twice the input."

Now write a one‑sentence summary of this function:""",
"code": '''
def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
'''
},
# 11 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (prompt from comment)
Comment: # add two numbers
Prompt: "Write a Python function that adds two numbers."

Generate a prompt (one sentence) that could have produced the following prime‑check function:""",
"code": '''
# This function checks if a number is prime
def is_prime(n):
    if n <= 1: return False
    for i in range(2,int(n**0.5)+1):
        if n % i == 0: return False
    return True
'''
},
# 12 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (bug explanation)
Factorial should return 1 when n==0; loop needs range(1,n+1).

Fix the factorial implementation below so factorial(0)==1 and factorial(5)==120:""",
"code": '''
def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result
'''
},
# 13 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (C linked‑list delete)
Maintain prev & curr pointers; update head if first node matches.

Implement `deleteNode` that removes the node whose data == key:""",
"code": '''
struct Node{ int data; struct Node* next; };

void deleteNode(struct Node** head, int key){
    // TODO
}
'''
},
# 14 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
Base cases: 0→0, 1→1; recurse n‑1 + n‑2.

Fill in the missing parts of fibonacci(n):""",
"code": '''
def fibonacci(n):
    # TODO
    pass
'''
},
# 15 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
class Car: def __init__(self,make): self.make=make

Add params name, age, optional email to the constructor below:""",
"code": '''
class Person:
    def __init__(self):
        pass
'''
},
# 16 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (binary search)
while(l<=r): mid=(l+r)//2; compare arr[mid] with target...

Complete the binarySearch method so it returns index or -1:""",
"code": '''
public int binarySearch(int[] arr,int target){
    int left=0,right=arr.length-1;
    while(left<=right){
        int mid=(left+right)/2;
        // TODO
    }
    return -1;
}
'''
},
# 17 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
Function name says isEven but body checks odd ➜ fix logic or rename.

Correct the inconsistency in the following C++ function:""",
"code": '''
bool isOdd(int x){
    return x % 2 == 0;
}
'''
},
# 18 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
Bug: returns 1/0 instead of boolean; fix with n%2==0.

Identify & repair the bug in this JS function:""",
"code": '''
function isEven(n){
    return n % 2;
}
'''
},
# 19 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
High‑level summary ➜ break into steps: validate → compute → return.

Decompose the comment and list each logical step of the next C++ function:""",
"code": '''
// Function that validates an input, calculates square, and returns result
int process(int x){
    if(x<0) return -1;
    return x*x;
}
'''
},
# 20 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
Compute average: total = sum(lst); return total/len(lst).

Finish calculate_average so it returns the mean of scores:""",
"code": '''
def calculate_average(scores):
    total = 0
    # TODO
    pass
'''
},
# 21 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example (refactor)
Add input validation, use pathlib, guard main, etc.

Review the utility script below, list issues, then provide a refactored version:""",
"code": '''
# utils.py ...
import csv
def read_csv(filepath):
    with open(filepath,'r') as f:
        return [row for row in csv.reader(f)]
def summarize_column(data,index):
    values=[float(row[index]) for row in data[1:]]
    total=sum(values); avg=total/len(values); return total,avg
def main():
    filepath='data.csv'
    data=read_csv(filepath)
    total,avg=summarize_column(data,1)
    print("Total:",total); print("Average:",avg)
if __name__=='__main__': main()
'''
},
# 22 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Example
clean_line: remove punctuation via str.translate, lower(), strip.

Complete the script so it cleans text and counts word frequencies:""",
"code": '''
# file_processor.py
import string
def load_file(fp): ...
def clean_line(line):
    # TODO
    pass
def count_words(lines): ...
'''
}
]


In [ ]:
tasks_self_consistency = [
# 1 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""You are a skilled Java analyst. Think step‑by‑step about what the method does.
After your reasoning, write
--- Final Answer:
followed by *one concise sentence* summarising the method.""",
"code": '''
public Map<String,Integer> countWordFrequency(List<String> words){
    Map<String,Integer> freqMap = new HashMap<>();
    for(String word:words){
        freqMap.put(word,freqMap.getOrDefault(word,0)+1);
    }
    return freqMap;
}
'''
},
# 2 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Spot the off‑by‑one bug. Reason through the loop bounds carefully.
End with
--- Final Answer:
and show the corrected Python code.""",
"code": '''
def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
'''
},
# 3 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Classify the main C++ bug type. Think aloud about lifetime and pointers.
Finish with
--- Final Answer:
<one‑phrase bug category>""",
"code": '''
int* getArray(int size){
    int arr[size];
    return arr;
}
'''
},
# 4 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Design a regex email validator. List key parts (username, @, domain, TLD).
Provide candidate pattern and a final function definition.
End with
--- Final Answer:
<completed Python function>""",
"code": '''
def is_valid_email(email):
    # TODO: Complete using regex
    pass
'''
},
# 5 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Create a Flask endpoint `/greet/<username>` that returns JSON.
Reason about route, parameter, jsonify call.
Conclude with
--- Final Answer:
<finished greet function>""",
"code": '''
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    # TODO
    pass
'''
},
# 6 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Design three SQL tables (users, books, reviews) with PK/FK constraints.
Explain choices, then write the DDL.
--- Final Answer:
<CREATE TABLE statements>""",
"code": '''
-- your schema here
'''
},
# 7 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Identify the null‑dereference risk and propose a safe implementation.
Think through possible inputs.
--- Final Answer:
<safe Java version>""",
"code": '''
public int getLength(String s){
    return s.length();
}
'''
},
# 8 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Improve CSV parsing to respect quoted fields.
Discuss pitfalls, then give finished code.
--- Final Answer:
<new parse_csv_line function>""",
"code": '''
def parse_csv_line(line):
    return line.split(',')  # broken
'''
},
# 9 ─────────────────────────────────────────────────────────────────────────────
{
"content":
"""Expose Product via Ktor: GET /product/{id}, POST /product.
Outline routing, then present the Ktor code block.
--- Final Answer:
<routes code>""",
"code": '''
data class Product(val id:Int, val name:String, val price:Double)
'''
},
# 10 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Explain step‑by‑step what the Python function does.
--- Final Answer:
<one‑sentence summary>""",
"code": '''
def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
'''
},
# 11 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Derive a suitable natural‑language prompt that could produce the given code.
Think about intent, then present.
--- Final Answer:
<single‑sentence prompt>""",
"code": '''
# This function checks if a number is prime
def is_prime(n):
    if n <= 1: return False
    for i in range(2,int(n**0.5)+1):
        if n % i == 0: return False
    return True
'''
},
# 12 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Find why factorial(0) fails, reason about ranges, fix the code.
--- Final Answer:
<corrected factorial function>""",
"code": '''
def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result
'''
},
# 13 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Plan node deletion in a singly‑linked list (handle head, middle, tail).
Give C implementation.
--- Final Answer:
<completed deleteNode>""",
"code": '''
struct Node{ int data; struct Node* next; };

void deleteNode(struct Node** head, int key){
    // TODO
}
'''
},
# 14 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Write fibonacci recursively. Show base cases, recursive formula.
--- Final Answer:
<full Python function>""",
"code": '''
def fibonacci(n):
    # TODO
    pass
'''
},
# 15 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Finish Person constructor with name, age, optional email.
Explain design, then show code.
--- Final Answer:
<class with __init__>""",
"code": '''
class Person:
    def __init__(self):
        pass
'''
},
# 16 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Complete binary search: think about mid update and return index.
--- Final Answer:
<completed method>""",
"code": '''
public int binarySearch(int[] arr,int target){
    int left=0,right=arr.length-1;
    while(left<=right){
        int mid=(left+right)/2;
        // TODO
    }
    return -1;
}
'''
},
# 17 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Resolve name/logic mismatch in isOdd.
Reason, then provide corrected C++ code.
--- Final Answer:
<fixed function>""",
"code": '''
bool isOdd(int x){
    return x % 2 == 0;
}
'''
},
# 18 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Identify bug in JS isEven and correct it to return boolean.
--- Final Answer:
<fixed JS function>""",
"code": '''
function isEven(n){
    return n % 2;
}
'''
},
# 19 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Break down the C++ function’s behaviour into atomic steps.
List them clearly.
--- Final Answer:
<ordered bullet list>""",
"code": '''
// Function that validates an input, calculates square, and returns result
int process(int x){
    if(x<0) return -1;
    return x*x;
}
'''
},
# 20 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Infer intent (average of scores) and finish implementation.
Walk through formula, then code.
--- Final Answer:
<completed calculate_average>""",
"code": '''
def calculate_average(scores):
    total = 0
    # TODO
    pass
'''
},
# 21 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Review util script: list issues (I/O, errors), then provide clean refactor.
--- Final Answer:
<refactored utils.py>""",
"code": '''
# utils.py - Script to parse and summarise numeric CSV files
import csv
def read_csv(filepath):
    with open(filepath,'r') as f:
        return [row for row in csv.reader(f)]
def summarize_column(data,index):
    values=[float(row[index]) for row in data[1:]]
    total=sum(values); avg=total/len(values); return total,avg
def main():
    filepath='data.csv'
    data=read_csv(filepath)
    total,avg=summarize_column(data,1)
    print("Total:",total); print("Average:",avg)
if __name__=='__main__': main()
'''
},
# 22 ────────────────────────────────────────────────────────────────────────────
{
"content":
"""Complete file_processor: clean punctuation (translate), lowercase, count.
Explain steps, then supply finished code.
--- Final Answer:
<full script>""",
"code": '''
# file_processor.py
import string
def load_file(fp): ...
def clean_line(line):
    # TODO
    pass
def count_words(lines): ...
'''
}
]


In [ ]:
tasks_prompt_chaining = [
# 1 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Read the Java method below and list, in bullets, every distinct operation it performs.",
"step2": "Using the analysis below:\n{analysis}\nWrite ONE concise sentence summarising the method.",
"code": '''
public Map<String,Integer> countWordFrequency(List<String> words){
    Map<String,Integer> freqMap = new HashMap<>();
    for(String word:words){
        freqMap.put(word,freqMap.getOrDefault(word,0)+1);
    }
    return freqMap;
}
'''
},
# 2 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Inspect the Python function and explain exactly why an off-by-one error occurs.",
"step2": "Based on the explanation:\n{analysis}\nProvide corrected code that sums *inclusive* of both start and end.",
"code": '''
def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
'''
},
# 3 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Examine the C++ function. Identify the bug category (e.g., UAF, buffer-overflow, logic).  Give a one-sentence justification.",
"step2": "With the identified category and justification:\n{analysis}\nReturn ONLY the phrase that names the bug category.",
"code": '''
int* getArray(int size){
    int arr[size];
    return arr;
}
'''
},
# 4 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Outline the pieces a regex email validator must test (username rules, @, domain, TLD).",
"step2": "Using the outline below:\n{analysis}\nWrite a Python function is_valid_email that returns True/False.",
"code": '''
def is_valid_email(email):
    # TODO
    pass
'''
},
# 5 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Describe the JSON structure a '/greet/<username>' Flask endpoint should return.",
"step2": "Following the description:\n{analysis}\nWrite the greet function so Flask returns that JSON.",
"code": '''
from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    # TODO
    pass
'''
},
# 6 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Draft the tables, primary keys and foreign keys needed for users, books, reviews.",
"step2": "Convert the draft below:\n{analysis}\ninto executable SQL CREATE TABLE statements.",
"code": '''
-- put schema here
'''
},
# 7 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Explain what can go wrong if s is null in the Java method.",
"step2": "Using that explanation:\n{analysis}\nwrite a safe implementation that avoids NullPointerException.",
"code": '''
public int getLength(String s){
    return s.length();
}
'''
},
# 8 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "List corner-cases a CSV line parser must handle when fields are quoted.",
"step2": "Refactor the function below using the corner-cases list:\n{analysis}\nso it returns an accurate list of fields.",
"code": '''
def parse_csv_line(line):
    return line.split(',')  # broken
'''
},
# 9 ─────────────────────────────────────────────────────────────────────────────
{
"step1": "Sketch the REST routes needed to expose Product via Ktor (URLs, verbs).",
"step2": "Turn the sketch below into Kotlin Ktor routing code:\n{analysis}",
"code": '''
data class Product(val id:Int, val name:String, val price:Double)
'''
},
# 10 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Break down what each line of the Python function does.",
"step2": "Using that breakdown:\n{analysis}\nwrite a one-sentence summary of the function’s purpose.",
"code": '''
def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
'''
},
# 11 ────────────────────────────────────────────────────────────────────────────
{
"step1": "From the comment and code, infer the programmer’s *intended English instruction*.",
"step2": "Rewrite the inferred instruction below:\n{analysis}\ninto ONE clear prompt for ChatGPT.",
"code": '''
# This function checks if a number is prime
def is_prime(n):
    if n <= 1: return False
    for i in range(2,int(n**0.5)+1):
        if n % i == 0: return False
    return True
'''
},
# 12 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Reason why factorial(0) currently returns 1 incorrectly. Show the loop logic.",
"step2": "Apply the reasoning:\n{analysis}\nand output corrected code.",
"code": '''
def factorial(n):
    result = 1
    for i in range(1, n):
        result *= i
    return result
'''
},
# 13 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Describe the algorithm to delete a node by value in a singly-linked list.",
"step2": "Implement deleteNode according to the algorithm:\n{analysis}",
"code": '''
struct Node{ int data; struct Node* next; };

void deleteNode(struct Node** head,int key){
    // TODO
}
'''
},
# 14 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Write the recurrence and base cases for Fibonacci recursion.",
"step2": "Using them:\n{analysis}\ncomplete the Python function.",
"code": '''
def fibonacci(n):
    # TODO
    pass
'''
},
# 15 ────────────────────────────────────────────────────────────────────────────
{
"step1": "List the attributes Person should initialise (name, age, email optional).",
"step2": "Implement __init__ below following the list:\n{analysis}",
"code": '''
class Person:
    def __init__(self):
        pass
'''
},
# 16 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Explain how classic binary search updates left/right based on arr[mid].",
"step2": "Fill in the TODOs using the explanation:\n{analysis}",
"code": '''
public int binarySearch(int[] arr,int target){
    int left=0,right=arr.length-1;
    while(left<=right){
        int mid=(left+right)/2;
        // TODO
    }
    return -1;
}
'''
},
# 17 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Identify the mismatch between function name isOdd and its logic.",
"step2": "Provide corrected code that resolves the mismatch, per analysis:\n{analysis}",
"code": '''
bool isOdd(int x){
    return x % 2 == 0;
}
'''
},
# 18 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Analyse why JS isEven returns 1|0 not boolean.",
"step2": "Fix the function based on that analysis:\n{analysis}",
"code": '''
function isEven(n){
    return n % 2;
}
'''
},
# 19 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Decompose the high-level comment into ordered steps the function performs.",
"step2": "Return the ordered list from analysis:\n{analysis}",
"code": '''
// Function that validates an input, calculates square, and returns result
int process(int x){
    if(x<0) return -1;
    return x*x;
}
'''
},
# 20 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Plan how to compute an average from a list (sum / len).",
"step2": "Implement calculate_average using plan:\n{analysis}",
"code": '''
def calculate_average(scores):
    total = 0
    # TODO
    pass
'''
},
# 21 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Critique the utility script: list at least three issues (error handling, etc.).",
"step2": "Refactor the script fully, addressing issues listed:\n{analysis}",
"code": '''
# utils.py - Script to parse and summarise numeric CSV files
import csv
def read_csv(filepath):
    with open(filepath,'r') as f:
        return [row for row in csv.reader(f)]
def summarize_column(data,index):
    values=[float(row[index]) for row in data[1:]]
    total=sum(values)
    avg=total/len(values)
    return total,avg
def main():
    filepath='data.csv'
    data=read_csv(filepath)
    total,avg=summarize_column(data,1)
    print("Total:",total)
    print("Average:",avg)
if __name__=='__main__':
    main()
'''
},
# 22 ────────────────────────────────────────────────────────────────────────────
{
"step1": "Outline steps to clean text (remove punctuation, lower-case) and count words.",
"step2": "Complete file_processor.py using the outline:\n{analysis}",
"code": '''
# file_processor.py - incomplete
import string
def load_file(fp): ...
def clean_line(line):
    # TODO
    pass
def count_words(lines): ...
'''
}
]


In [ ]:
zero_shot = pd.DataFrame(zero_shot_tasks)
zero_shot = zero_shot.rename(columns={'content': 'prompt'})
zero_shot.insert(2, 'model1_response', '')
zero_shot.insert(3, 'model2_response', '')


zero_shot



,prompt,code,model1_response,model2_response
0,Summarize the functionality of the following m...,"\npublic Map<String, Integer> countWordFrequen...",,
1,Identify and fix the off-by-one error in this ...,"\ndef sum_range(start, end):\n total = 0\n ...",,
2,Classify the type of bug in the following C++ ...,\nint* getArray(int size) {\n int arr[size]...,,
3,Complete the function using regex to validate ...,\ndef is_valid_email(email):\n # TODO: Comp...,,
4,Create a ‘/greet/<username>’ endpoint that ret...,"\nfrom flask import Flask, jsonify\n\napp = Fl...",,
5,"Write the schema for a review app with users, ...",\n-- TODO: Design schema with appropriate keys...,,
6,Identify any null dereference risk:,\npublic int getLength(String s) {\n return...,,
7,Improve the parser to support quoted fields:,\ndef parse_csv_line(line):\n return line.s...,,
8,Convert the data class to a REST API using Ktor:,"\ndata class Product(val id: Int, val name: St...",,
9,Write a brief summary of the function:,\ndef reverse_words(sentence):\n return ' '...,,


In [ ]:
chain_of_thought = pd.DataFrame(chain_of_thought_tasks)
chain_of_thought = chain_of_thought.rename(columns={'content': 'prompt'})
chain_of_thought.insert(2, 'model1_response', '')
chain_of_thought.insert(3, 'model2_response', '')
chain_of_thought

,prompt,code,model1_response,model2_response
0,Let's break down the method step-by-step. Firs...,"\npublic Map<String, Integer> countWordFrequen...",,
1,Let's first analyze the loop. The range for i ...,"\ndef sum_range(start, end):\n total = 0\n ...",,
2,Let's walk through this function. The function...,\nint* getArray(int size) {\n int arr[size]...,,
3,"To validate an email address using regex, we n...",\ndef is_valid_email(email):\n # TODO: Comp...,,
4,"First, think about the endpoint '/greet/<usern...","\nfrom flask import Flask, jsonify\n\napp = Fl...",,
5,Let's start by defining the tables needed for ...,\n-- TODO: Design schema with appropriate keys...,,
6,The function getLength() tries to access the l...,\npublic int getLength(String s) {\n return...,,
7,"Currently, the function splits each line by co...",\ndef parse_csv_line(line):\n return line.s...,,
8,We need to expose a Product data class through...,"\ndata class Product(val id: Int, val name: St...",,
9,Let's break down this function. It splits the ...,\ndef reverse_words(sentence):\n return ' '...,,


In [ ]:
few_shot = pd.DataFrame(tasks_few_shot)
few_shot = few_shot.rename(columns={'content': 'prompt'})
few_shot.insert(2, 'model1_response', '')
few_shot.insert(3, 'model2_response', '')
few_shot

,prompt,code,model1_response,model2_response
0,"Example 1 (Java)\nCode:\npublic int add(int a,...","\npublic Map<String,Integer> countWordFrequenc...",,
1,Example (Python)\nBuggy code:\ndef inclusive_s...,"\ndef sum_range(start, end):\n total = 0\n ...",,
2,Example (C++)\nBuggy function:\nint* bad(){ in...,\nint* getArray(int size){\n int arr[size];...,,
3,Example (Python + regex)\nInput regex: r'^[A-Z...,\ndef is_valid_email(email):\n # TODO: Comp...,,
4,Example (Flask)\n@app.route('/hello/<name>')\n...,"\nfrom flask import Flask, jsonify\napp = Flas...",,
5,Example (SQL schema snippet)\nCREATE TABLE use...,\n-- your schema here\n,,
6,Example (Java null‑check)\nif(s == null) retur...,\npublic int getLength(String s){\n return ...,,
7,Example (Python CSV)\nUse csv module or regex ...,\ndef parse_csv_line(line):\n return line.s...,,
8,"Example (Ktor)\nget(""/product/{id}""){ ... }\np...","\ndata class Product(val id:Int, val name:Stri...",,
9,Example (summary)\n`def double(x): return 2*x`...,\ndef reverse_words(sentence):\n return ' '...,,


In [ ]:
self_consistency = pd.DataFrame(tasks_self_consistency)
self_consistency = self_consistency.rename(columns={'content': 'prompt'})
self_consistency.insert(2, 'model1_response', '')
self_consistency.insert(3, 'model2_response', '')
self_consistency

,prompt,code,model1_response,model2_response
0,You are a skilled Java analyst. Think step‑by‑...,"\npublic Map<String,Integer> countWordFrequenc...",,
1,Spot the off‑by‑one bug. Reason through the lo...,"\ndef sum_range(start, end):\n total = 0\n ...",,
2,Classify the main C++ bug type. Think aloud ab...,\nint* getArray(int size){\n int arr[size];...,,
3,Design a regex email validator. List key parts...,\ndef is_valid_email(email):\n # TODO: Comp...,,
4,Create a Flask endpoint `/greet/<username>` th...,"\nfrom flask import Flask, jsonify\napp = Flas...",,
5,"Design three SQL tables (users, books, reviews...",\n-- your schema here\n,,
6,Identify the null‑dereference risk and propose...,\npublic int getLength(String s){\n return ...,,
7,Improve CSV parsing to respect quoted fields.\...,\ndef parse_csv_line(line):\n return line.s...,,
8,"Expose Product via Ktor: GET /product/{id}, PO...","\ndata class Product(val id:Int, val name:Stri...",,
9,Explain step‑by‑step what the Python function ...,\ndef reverse_words(sentence):\n return ' '...,,


In [ ]:
prompt_chaining = pd.DataFrame(tasks_prompt_chaining)
prompt_chaining.insert(3, 'model1_response_step1', '')
prompt_chaining.insert(4, 'model1_response_step2', '')
prompt_chaining.insert(5, 'model2_response_step1', '')
prompt_chaining.insert(6, 'model2_response_step2', '')
prompt_chaining

,step1,step2,code,model1_response_step1,model1_response_step2,model2_response_step1,model2_response_step2
0,"Read the Java method below and list, in bullet...",Using the analysis below:\n{analysis}\nWrite O...,"\npublic Map<String,Integer> countWordFrequenc...",,,,
1,Inspect the Python function and explain exactl...,Based on the explanation:\n{analysis}\nProvide...,"\ndef sum_range(start, end):\n total = 0\n ...",,,,
2,Examine the C++ function. Identify the bug cat...,With the identified category and justification...,\nint* getArray(int size){\n int arr[size];...,,,,
3,Outline the pieces a regex email validator mus...,Using the outline below:\n{analysis}\nWrite a ...,\ndef is_valid_email(email):\n # TODO\n ...,,,,
4,Describe the JSON structure a '/greet/<usernam...,Following the description:\n{analysis}\nWrite ...,"\nfrom flask import Flask, jsonify\napp = Flas...",,,,
5,"Draft the tables, primary keys and foreign key...",Convert the draft below:\n{analysis}\ninto exe...,\n-- put schema here\n,,,,
6,Explain what can go wrong if s is null in the ...,Using that explanation:\n{analysis}\nwrite a s...,\npublic int getLength(String s){\n return ...,,,,
7,List corner-cases a CSV line parser must handl...,Refactor the function below using the corner-c...,\ndef parse_csv_line(line):\n return line.s...,,,,
8,Sketch the REST routes needed to expose Produc...,Turn the sketch below into Kotlin Ktor routing...,"\ndata class Product(val id:Int, val name:Stri...",,,,
9,Break down what each line of the Python functi...,Using that breakdown:\n{analysis}\nwrite a one...,\ndef reverse_words(sentence):\n return ' '...,,,,


In [ ]:
# Loop through each task in the list and generate responses using the API.
count = 0
for task in zero_shot_tasks:
    count += 1
    print(f"Question {count}:\n")
    code = task["code"]
    content = task["content"]

    response1 = client.chat.completions.create(
        model = model_choices[2],  # OpenAI GPT-4.1
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    zero_shot.loc[count-1, 'model1_response'] = response1.choices[0].message.content
    print(response1.choices[0].message.content)
    print("\n\n***************************************************************************\n\n")

    # Second model response
    response2 = client.chat.completions.create(
        model = model_choices[3],  # Codestral-2501
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    zero_shot.loc[count-1, 'model2_response'] = response2.choices[0].message.content
    print(response2.choices[0].message.content)
    print("\n\n***************************************************************************NEXT QUESTION***************************************************************************\n\n")
    #break

Question 1:

This method `countWordFrequency` takes a list of strings (`words`) as input and returns a map where:

- The **keys** are the unique words from the list.
- The **values** are the counts of how many times each word appears in the list.

In other words, it counts the frequency of each word in the input list and returns a mapping of each word to its occurrence count.


***************************************************************************


The method `countWordFrequency` takes a list of words and returns a map that records the number of occurrences (frequency) of each unique word. It iterates through the list, updating the count for each word in the map, and finally returns the map containing each word and its corresponding frequency count.


***************************************************************************NEXT QUESTION***************************************************************************


Question 2:

The current function uses `range(start, end)`, which 

In [ ]:
zero_shot

,prompt,code,model1_response,model2_response
0,Summarize the functionality of the following m...,"\npublic Map<String, Integer> countWordFrequen...",**Summary of method functionality:**\n\nThe `c...,The method `countWordFrequency` takes a list o...
1,Identify and fix the off-by-one error in this ...,"\ndef sum_range(start, end):\n total = 0\n ...",Let's analyze the function:\n\n```python\ndef ...,The off-by-one error in the function `sum_rang...
2,Classify the type of bug in the following C++ ...,\nint* getArray(int size) {\n int arr[size]...,"The bug in the provided C++ function is a **""d...",The function `int* getArray(int size)` contain...
3,Complete the function using regex to validate ...,\ndef is_valid_email(email):\n # TODO: Comp...,Certainly! Here’s a simple way to use regex to...,To validate basic email addresses using regula...
4,Create a ‘/greet/<username>’ endpoint that ret...,"\nfrom flask import Flask, jsonify\n\napp = Fl...",Certainly! You can use Flask's `jsonify` funct...,Certainly! You can create a `/greet/<username>...
5,"Write the schema for a review app with users, ...",\n-- TODO: Design schema with appropriate keys...,Certainly! Here’s a complete SQL schema for yo...,"Sure, here is a sample schema for a review app..."
6,Identify any null dereference risk:,\npublic int getLength(String s) {\n return...,"Yes, **there is a null dereference risk** in t...",The code snippet you provided has a potential ...
7,Improve the parser to support quoted fields:,\ndef parse_csv_line(line):\n return line.s...,Certainly! Handling quoted fields in CSV requi...,To improve the parser to handle quoted fields ...
8,Convert the data class to a REST API using Ktor:,"\ndata class Product(val id: Int, val name: St...",Certainly! Here’s how you can expose your `Pro...,To create a REST API using Ktor with GET and P...
9,Write a brief summary of the function:,\ndef reverse_words(sentence):\n return ' '...,The `reverse_words` function takes a string `s...,The function `reverse_words(sentence)` takes a...


In [ ]:
# Loop through each task in the list and generate responses using the API.
count = 0
for task in chain_of_thought_tasks:
    count += 1
    code = task["code"]
    content = task["content"]

    response1 = client.chat.completions.create(
        model = model_choices[2],  # OpenAI GPT-4.1
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    chain_of_thought.loc[count-1, 'model1_response'] = response1.choices[0].message.content

    # Second model response
    response2 = client.chat.completions.create(
        model = model_choices[3],  # Codestral-2501
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    chain_of_thought.loc[count-1, 'model2_response'] = response2.choices[0].message.content
    print(f"✓ finished question {count-1+1:02d}")
print("All tasks completed — dataframe updated!")

✓ finished question 01
✓ finished question 02
✓ finished question 03
✓ finished question 04
✓ finished question 05
✓ finished question 06
✓ finished question 07
✓ finished question 08
✓ finished question 09
✓ finished question 10
✓ finished question 11
✓ finished question 12
✓ finished question 13
✓ finished question 14
✓ finished question 15
✓ finished question 16
✓ finished question 17
✓ finished question 18
✓ finished question 19
✓ finished question 20
✓ finished question 21
✓ finished question 22
All tasks completed — dataframe updated!


In [ ]:
chain_of_thought

,prompt,code,model1_response,model2_response
0,Let's break down the method step-by-step. Firs...,"\npublic Map<String, Integer> countWordFrequen...",The method counts and returns the frequency of...,This method counts the frequency of each word ...
1,Let's first analyze the loop. The range for i ...,"\ndef sum_range(start, end):\n total = 0\n ...",You're right that the current loop runs from `...,Your current loop:\n\n```python\nfor i in rang...
2,Let's walk through this function. The function...,\nint* getArray(int size) {\n int arr[size]...,The bug here is **returning a pointer to a loc...,The bug in this function is **returning a poin...
3,"To validate an email address using regex, we n...",\ndef is_valid_email(email):\n # TODO: Comp...,Here's a Python function that uses a regex pat...,Certainly! To validate an email address using ...
4,"First, think about the endpoint '/greet/<usern...","\nfrom flask import Flask, jsonify\n\napp = Fl...","```python\nfrom flask import Flask, jsonify\n\...",Certainly! To return a JSON response with a gr...
5,Let's start by defining the tables needed for ...,\n-- TODO: Design schema with appropriate keys...,```sql\nCREATE TABLE users (\n id SERIAL PR...,Certainly! Here's a well-structured schema for...
6,The function getLength() tries to access the l...,\npublic int getLength(String s) {\n return...,To avoid a `NullPointerException` when `s` is ...,To avoid a NullPointerException when `s` is nu...
7,"Currently, the function splits each line by co...",\ndef parse_csv_line(line):\n return line.s...,To properly parse a CSV line that may contain ...,To correctly handle quoted fields that may con...
8,We need to expose a Product data class through...,"\ndata class Product(val id: Int, val name: St...",Certainly! Here's a complete example of a simp...,Certainly! Below is a complete example of how ...
9,Let's break down this function. It splits the ...,\ndef reverse_words(sentence):\n return ' '...,The function `reverse_words` takes a sentence ...,"The function takes a sentence, reverses the or..."


In [ ]:
# Loop through each task in the list and generate responses using the API.
count = 0
for task in tasks_few_shot:
    count += 1
    print(f"Question {count}:\n")
    code = task["code"]
    content = task["content"]

    # OpenAI GPT-4 response
    # print(f"code = '''\n{code}'''")

    response1 = client.chat.completions.create(
        model = model_choices[2],  # OpenAI GPT-4.1
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    few_shot.loc[count-1, 'model1_response'] = response1.choices[0].message.content
    print(response1.choices[0].message.content)
    print("\n\n***************************************************************************\n\n")

    # Second model response
    response2 = client.chat.completions.create(
        model = model_choices[3],  # Codestral-2501
        messages=[
            {"role": "user",
             "content": f"{content}\n{code}"
            },
        ],
        max_tokens=1024,
        temperature=0.7,
    )

    few_shot.loc[count-1, 'model2_response'] = response2.choices[0].message.content
    print(response2.choices[0].message.content)
    print("\n\n***************************************************************************NEXT QUESTION***************************************************************************\n\n")
    #break

Question 1:

Counts the frequency of each word in a list and returns a map of words to their occurrence counts.


***************************************************************************


Counts and returns a map of each word's frequency in the input list.


***************************************************************************NEXT QUESTION***************************************************************************


Question 2:

The function is intended to sum all integers from `start` to `end` **inclusive**, but `range(start, end)` excludes `end`. This causes an off-by-one error by missing the last number.

**Fix:** Use `range(start, end + 1)` to include the `end` value in the sum.

```python
def sum_range(start, end):
    total = 0
    for i in range(start, end + 1):
        total += i
    return total
```


***************************************************************************


The off-by-one error occurs because `range(start, end)` excludes `end`, so the sum does not

In [ ]:
few_shot

In [ ]:
import collections
import re

# -------- helper utilities ----------------------------------------------------
FINAL_DIVIDER = r'---\s*Final\s*Answer\s*:?'      # the divider string we put in every prompt

def extract_final(text: str) -> str:
    """
    Pull everything that appears after the final‑answer divider.
    Falls back to the full text if the divider isn't found.
    """
    m = re.split(FINAL_DIVIDER, text, flags=re.IGNORECASE)
    return m[-1].strip() if len(m) > 1 else text.strip()


def majority_final_answer(task: dict,
                          model_name: str,
                          reps: int = 3,
                          temperature: float = 0.7) -> tuple[str, int]:
    """
    Run the prompt `reps` times, grab the final answer portion,
    and return (most_common_answer, frequency).
    """
    answers = []

    for _ in range(reps):
        resp = client.chat.completions.create(
            model=model_name,
            messages=[{
                "role": "user",
                "content": f"{task['content']}\n{task['code']}"
            }],
            temperature=temperature,
            max_tokens=1024,
        )
        answers.append(extract_final(resp.choices[0].message.content))

    best, freq = collections.Counter(answers).most_common(1)[0]
    return best, freq
# -----------------------------------------------------------------------------


# assume `tasks_self_consistency` is your list of 22 tasks
# and `self_consistency` is the dataframe you showed (index 0‑21, cols prompt/code/…)
for idx, task in enumerate(tasks_self_consistency):

    # ------------  GPT-4.1 with self‑consistency  -----------------------------
    maj_answer, votes = majority_final_answer(
        task,
        model_name=model_choices[2],   # GPT-4.1
        reps=3,                        # number of reasoning samples
        temperature=0.7                # make sampling diverse
    )
    self_consistency.loc[idx, 'model1_response'] = f"(votes {votes}/3)\n{maj_answer}"

    # ------------  Codestral 25.01 with self-consistency  ---------------------
    maj_answer, votes = majority_final_answer(
        task,
        model_name=model_choices[3],   # Codestral 25.01
        reps=3,                        # number of reasoning samples
        temperature=0.7                # make sampling diverse
    )
    self_consistency.loc[idx, 'model2_response'] = f"(votes {votes}/3)\n{maj_answer}"

    print(f"✓ finished question {idx+1:02d}")

print("All tasks completed — dataframe updated!")


✓ finished question 01
✓ finished question 02
✓ finished question 03
✓ finished question 04
✓ finished question 05
✓ finished question 06
✓ finished question 07
✓ finished question 08
✓ finished question 09
✓ finished question 10
✓ finished question 11
✓ finished question 12
✓ finished question 13
✓ finished question 14
✓ finished question 15
✓ finished question 16
✓ finished question 17
✓ finished question 18
✓ finished question 19
✓ finished question 20
✓ finished question 21
✓ finished question 22
All tasks completed — dataframe updated!


In [ ]:
self_consistency

In [ ]:
for idx, task in enumerate(tasks_prompt_chaining):

    code, step1, step2 = task["code"], task["step1"], task["step2"]

    # ─────────── Model-1 (GPT-4o) ───────────
    msgs = [{"role": "user", "content": f"{step1}\n{code}"}]
    resp1_step1 = client.chat.completions.create(
        model=model_choices[0],
        messages=msgs,
        max_tokens=512,
        temperature=0.7,
    ).choices[0].message.content.strip()

    prompt_chaining.loc[idx, "model1_response_step1"] = resp1_step1

    final_prompt = step2.replace("{analysis}", resp1_step1) + f"\n{code}"
    msgs += [
        {"role": "assistant", "content": resp1_step1},
        {"role": "user",      "content": final_prompt},
    ]
    resp1_step2 = client.chat.completions.create(
        model=model_choices[0],
        messages=msgs,
        max_tokens=1024,
        temperature=0.7,
    ).choices[0].message.content.strip()

    prompt_chaining.loc[idx, "model1_response_step2"] = resp1_step2


    # ─────────── Model-2 (Codestral-2501) ───────────
    msgs2 = [{"role": "user", "content": f"{step1}\n{code}"}]
    resp2_step1 = client.chat.completions.create(
        model=model_choices[1],
        messages=msgs2,
        max_tokens=512,
        temperature=0.7,
    ).choices[0].message.content.strip()

    prompt_chaining.loc[idx, "model2_response_step1"] = resp2_step1

    final_prompt2 = step2.replace("{analysis}", resp2_step1) + f"\n{code}"
    msgs2 += [
        {"role": "assistant", "content": resp2_step1},
        {"role": "user",      "content": final_prompt2},
    ]
    resp2_step2 = client.chat.completions.create(
        model=model_choices[1],
        messages=msgs2,
        max_tokens=1024,
        temperature=0.7,
    ).choices[0].message.content.strip()

    prompt_chaining.loc[idx, "model2_response_step2"] = resp2_step2

    print(f"✓ finished question {idx+1:02d}")

print("All tasks completed — dataframe updated!")


✓ finished question 01
✓ finished question 02
✓ finished question 03
✓ finished question 04
✓ finished question 05
✓ finished question 06
✓ finished question 07
✓ finished question 08
✓ finished question 09
✓ finished question 10
✓ finished question 11
✓ finished question 12
✓ finished question 13
✓ finished question 14
✓ finished question 15
✓ finished question 16
✓ finished question 17
✓ finished question 18
✓ finished question 19
✓ finished question 20
✓ finished question 21
✓ finished question 22
All tasks completed — dataframe updated!


In [ ]:
prompt_chaining

,step1,step2,code,model1_response_step1,model1_response_step2,model2_response_step1,model2_response_step2
0,"Read the Java method below and list, in bullet...",Using the analysis below:\n{analysis}\nWrite O...,"\npublic Map<String,Integer> countWordFrequenc...",**Distinct operations performed by the method:...,This method takes a list of words and returns ...,Certainly! Here is a list of the distinct oper...,The method `countWordFrequency` iterates throu...
1,Inspect the Python function and explain exactl...,Based on the explanation:\n{analysis}\nProvide...,"\ndef sum_range(start, end):\n total = 0\n ...",Let's analyze the function:\n\n```python\ndef ...,Certainly! To make the function sum **inclusiv...,The Python function `sum_range` is designed to...,To fix the off-by-one error and ensure that th...
2,Examine the C++ function. Identify the bug cat...,With the identified category and justification...,\nint* getArray(int size){\n int arr[size];...,**Bug category:** Use-After-Free (UAF) / Dangl...,Use-After-Free (UAF),The bug category is logic. The function attemp...,logic
3,Outline the pieces a regex email validator mus...,Using the outline below:\n{analysis}\nWrite a ...,\ndef is_valid_email(email):\n # TODO\n ...,"Sure! To validate an email address via regex, ...",Certainly! Here’s a Python function `is_valid_...,Creating a regex email validator involves ensu...,Sure! Let's refine the `is_valid_email` functi...
4,Describe the JSON structure a '/greet/<usernam...,Following the description:\n{analysis}\nWrite ...,"\nfrom flask import Flask, jsonify\napp = Flas...",Certainly! A typical JSON structure for a `/gr...,Certainly! Here is the complete `greet` functi...,The JSON structure for the `/greet/<username>`...,Sure! Below is the implementation of the `gree...
5,"Draft the tables, primary keys and foreign key...",Convert the draft below:\n{analysis}\ninto exe...,\n-- put schema here\n,Certainly! Here’s a simple relational schema f...,```sql\nCREATE TABLE users (\n user_id SERI...,Certainly! Below is an example of a database s...,Certainly! Below are the executable SQL `CREAT...
6,Explain what can go wrong if s is null in the ...,Using that explanation:\n{analysis}\nwrite a s...,\npublic int getLength(String s){\n return ...,If `s` is `null` in the Java method\n\n```java...,Certainly! Here is a **safe implementation** o...,If the string `s` is `null` in the Java method...,To write a safe implementation of the `getLeng...
7,List corner-cases a CSV line parser must handl...,Refactor the function below using the corner-c...,\ndef parse_csv_line(line):\n return line.s...,Here are **corner-cases** a CSV line parser mu...,Here is a **refactored CSV line parser** that ...,When parsing CSV lines where fields are quoted...,To refactor the `parse_csv_line` function to h...
8,Sketch the REST routes needed to expose Produc...,Turn the sketch below into Kotlin Ktor routing...,"\ndata class Product(val id:Int, val name:Stri...",Certainly! For a typical REST API for managing...,Here is a complete **Kotlin Ktor routing imple...,"To expose a `Product` resource using Ktor, you...",To implement the RESTful routes for the `Produ...
9,Break down what each line of the Python functi...,Using that breakdown:\n{analysis}\nwrite a one...,\ndef reverse_words(sentence):\n return ' '...,Certainly! Here's a line-by-line breakdown of ...,The `reverse_words` function takes a sentence ...,"Sure, let's break down each line of the `rever...","The `reverse_words` function takes a sentence,..."


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zero_shot.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/zero-shot-raw-data.csv', index=False)

In [ ]:


chain_of_thought.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/chain-of-thought-raw-data.csv', index=False)

In [ ]:
few_shot.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/few-shot-raw-data.csv', index=False)

In [ ]:
self_consistency.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/self-consistency-raw-data.csv', index=False)

In [ ]:
prompt_chaining.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/prompt-chaining-raw-data.csv', index=False)

In [2]:
!pip install -q sacrebleu evaluate rouge-score nltk \
               sentence-transformers codebleu tree_sitter_languages
!pip install tree-sitter-python==0.21
!pip install textdistance
import nltk, evaluate, sacrebleu, pandas as pd, numpy as np
from sentence_transformers import SentenceTransformer, util
from codebleu import calc_codebleu
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
df = pd.read_csv("/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/Copy of chain-of-thought-raw-data.csv")      # adjust filename
df.head()


,prompt,code,model1_response,model2_response
0,Let's break down the method step-by-step. Firs...,"\npublic Map<String, Integer> countWordFrequen...",The `countWordFrequency` method takes a list o...,The method `countWordFrequency` takes a list o...
1,Let's first analyze the loop. The range for i ...,"\ndef sum_range(start, end):\n total = 0\n ...","You're correct! In Python, the `range(start, e...",The current implementation of the `sum_range` ...
2,Let's walk through this function. The function...,\nint* getArray(int size) {\n int arr[size]...,The bug in the function you've shown is known ...,The bug in the function `getArray` is related ...
3,"To validate an email address using regex, we n...",\ndef is_valid_email(email):\n # TODO: Comp...,Certainly! Here’s a simple Python function usi...,"To validate an email address using regex, you ..."
4,"First, think about the endpoint '/greet/<usern...","\nfrom flask import Flask, jsonify\n\napp = Fl...",Certainly! To return a JSON response with a gr...,"Sure, you can use the `jsonify` function from ..."


In [57]:
bleu      = evaluate.load("bleu")
rouge     = evaluate.load("rouge")
meteor    = evaluate.load("meteor")

def symmetric_overlap(row):
    refs = [row["model2_response"]]
    cand = [row["model1_response"]]
    bleu12  = bleu.compute(predictions=cand, references=refs)["bleu"]
    rouge12 = rouge.compute(predictions=cand, references=refs)["rougeL"]
    meteor12= meteor.compute(predictions=cand, references=refs)["meteor"]

    refs, cand = cand, refs                            # swap
    bleu21  = bleu.compute(predictions=cand, references=refs)["bleu"]
    rouge21 = rouge.compute(predictions=cand, references=refs)["rougeL"]
    meteor21= meteor.compute(predictions=cand, references=refs)["meteor"]

    return pd.Series({"BLEU":   (bleu12+bleu21)/2,
                      "ROUGE_L":(rouge12+rouge21)/2,
                      "METEOR": (meteor12+meteor21)/2})

df_overlap = df.apply(symmetric_overlap, axis=1)

#-------------------------------------------------------------------------------

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

def cosine(row):
    emb1 = model.encode(row["model1_response"], convert_to_tensor=True)
    emb2 = model.encode(row["model2_response"], convert_to_tensor=True)
    return util.cos_sim(emb1, emb2).item()

df["cosine_sim"] = df.apply(cosine, axis=1)

#-------------------------------------------------------------------------------

def codebleu_score(row, lang="python"):
    return calc_codebleu([row["model2_response"]], [row["model1_response"]], lang)

df["codebleu"] = df.apply(codebleu_score, axis=1)

#-------------------------------------------------------------------------------

df["exact_match"] = (df["model1_response"].str.strip() == df["model2_response"].str.strip()).astype(int)

import textdistance as td   # pip install textdistance
df["levenshtein"] = df.apply(lambda r: td.levenshtein(r["model1_response"], r["model2_response"]), axis=1)

#-------------------------------------------------------------------------------

report = pd.DataFrame({
    "BLEU"       : df_overlap["BLEU"].mean(),
    "ROUGE-L"    : df_overlap["ROUGE_L"].mean(),
    "METEOR"     : df_overlap["METEOR"].mean(),
    "Cosine"     : df["cosine_sim"].mean(),
    "CodeBLEU"   : df["codebleu"].str['codebleu'].mean(),
    "Exact-match": df["exact_match"].mean()*100,
    "Avg Edit Dist": df["levenshtein"].mean()
}, index=["Average over all rows"]).T

print(report.round(4))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


               Average over all rows
BLEU                          0.2568
ROUGE-L                       0.3591
METEOR                        0.3897
Cosine                        0.8962
CodeBLEU                      0.2983
Exact-match                   0.0000
Avg Edit Dist              1028.5455


In [58]:
df.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/chain-of-thought-analysis.csv', index=False)
report.to_csv('/content/drive/Shareddrives/CSCI_420/quang_assignment3/raw_data/chain-of-thought-report.csv')

In [28]:
report

,Average over all rows
BLEU,0.233237
ROUGE-L,0.354671
METEOR,0.389060
Cosine,0.886531
CodeBLEU,0.284266
Exact-match,0.000000
Avg Edit Dist,1137.590909


In [30]:
df

,prompt,code,model1_response,model2_response,cosine_sim,codebleu,exact_match,levenshtein
0,Summarize the functionality of the following m...,"\npublic Map<String, Integer> countWordFrequen...",**Summary of method functionality:**\n\nThe `c...,The method `countWordFrequency` takes a list o...,0.937718,"{'codebleu': 0.20009822168307242, 'ngram_match...",0,752
1,Identify and fix the off-by-one error in this ...,"\ndef sum_range(start, end):\n total = 0\n ...",Let's analyze the function:\n\n```python\ndef ...,The off-by-one error in the function `sum_rang...,0.883532,"{'codebleu': 0.26933157213979475, 'ngram_match...",0,850
2,Classify the type of bug in the following C++ ...,\nint* getArray(int size) {\n int arr[size]...,"The bug in the provided C++ function is a **""d...",The function `int* getArray(int size)` contain...,0.889291,"{'codebleu': 0.2579458106292575, 'ngram_match_...",0,1882
3,Complete the function using regex to validate ...,\ndef is_valid_email(email):\n # TODO: Comp...,Certainly! Here’s a simple way to use regex to...,To validate basic email addresses using regula...,0.890297,"{'codebleu': 0.19172574277139326, 'ngram_match...",0,1314
4,Create a ‘/greet/<username>’ endpoint that ret...,"\nfrom flask import Flask, jsonify\n\napp = Fl...",Certainly! You can use Flask's `jsonify` funct...,Certainly! You can create a `/greet/<username>...,0.939625,"{'codebleu': 0.1882375293027663, 'ngram_match_...",0,706
5,"Write the schema for a review app with users, ...",\n-- TODO: Design schema with appropriate keys...,Certainly! Here’s a complete SQL schema for yo...,"Sure, here is a sample schema for a review app...",0.887843,"{'codebleu': 0.33564828821105175, 'ngram_match...",0,1195
6,Identify any null dereference risk:,\npublic int getLength(String s) {\n return...,"Yes, **there is a null dereference risk** in t...",The code snippet you provided has a potential ...,0.896731,"{'codebleu': 0.4400229365464774, 'ngram_match_...",0,655
7,Improve the parser to support quoted fields:,\ndef parse_csv_line(line):\n return line.s...,Certainly! Handling quoted fields in CSV requi...,To improve the parser to handle quoted fields ...,0.830137,"{'codebleu': 0.20493909407657035, 'ngram_match...",0,1109
8,Convert the data class to a REST API using Ktor:,"\ndata class Product(val id: Int, val name: St...",Certainly! Here’s how you can expose your `Pro...,To create a REST API using Ktor with GET and P...,0.932964,"{'codebleu': 0.32382214080649846, 'ngram_match...",0,2141
9,Write a brief summary of the function:,\ndef reverse_words(sentence):\n return ' '...,The `reverse_words` function takes a string `s...,The function `reverse_words(sentence)` takes a...,0.958223,"{'codebleu': 0.26334689813888534, 'ngram_match...",0,379
